In [0]:
import numpy as np
import matplotlib.pyplot as plt
import time
import urllib
%matplotlib inline

In [0]:
#code written by us/ defining the points and visualizing the input data 
start_time = time.clock()
#Importing Dataset
urllib.request.urlretrieve(url = "http://people.sc.fsu.edu/~jburkardt/datasets/tsp/att48_xy.txt", filename = "tsp1.txt")
fname = 'tsp1.txt'
data= np.loadtxt(fname)
x=data[:,0]
y=data[:,1]
N = len(x) #Number of Cities

d = np.zeros([N,N]) # create a matrix with NxN martix

for i in range(N):
    for j in range(N):
        d[i, j]=np.sqrt(pow(x[i]-x[j],2)+pow(y[i]-y[j],2)) #Euclidean distance Matrix
        
#plt.plot(x, y, '-o')

In [0]:
# Calcualting the desired distance/ code written by us
d1 = np.zeros([N])
for i in range(N-1):
    d1[i]=np.sqrt((x[i]-x[i+1])**2+(y[i]-y[i+1])**2)

h=sum(d1)
h=h+np.sqrt((x[N-1]-x[0])**2+(y[N-1]-y[0])**2)
print(round(h,4))

157530.2462


In [0]:
# Parameters for network energy function / tuning the hyperparameter
A=500; B=500; C=1000; D=500; alpha=0.01

In [0]:
#network build and contraints taken from github with some minor changes
# x-v-value of each node, y-u-input potential, u0-gamma
def hopfield():
    u0=0.02
    toend=0
    udao=np.zeros([N, N])
    ctr=0
    while toend==0:
        ctr+=1
        # U initialization
        v = np.random.rand(N,N)
        u = np.ones([N, N])*(-u0*np.log(N-1)/2)
        u+=u*0.91
        for _ in range(1000):
            for vx in range(N):
                for vi in range(N):
                    j1,j2,j3,j4=0,0,0,0
                    # Constraint 1 (sum over columns j!=vi)
                    for j in range(N):
                        if j!=vi:
                            j1+=v[vx, j]
                            #print(j, vi, j1)
                    j1*=-A
                    # contraint 2 (sum over rows y!=x)
                    for y in range(N):
                        if y!=vx:
                            j2+=v[y, vi]
                    j2*=-B
                    # constraint 3 (overall sum)
                    j3 = np.sum(v)
                    j3=-C*(j3-N)

                    # Shortest distance contraint 
                    for y in range(N):
                        if y!=vx:
                            if vi==0:
                                j4+=d[vx, y]*(v[y, vi+1]+v[y, N-1])
                            elif vi==N-1:
                                j4+=d[vx, y]*(v[y, vi-1]+v[y, 0])
                            else:
                                j4+=d[vx, y]*(v[y, vi+1]+v[y, vi-1])
                    j4*=-D
                    udao[vx, vi]= -u[vx, vi]+j1+j2+j3+j4
            # update status and derivatives
            u=u+alpha*udao
            # calculate node value from input potential u
            v=(1+np.tanh(u/u0))/2
            # threshold
            for vx in range(N):
                for vi in range(N):
                    if(v[vx, vi]<0.7):
                        v[vx, vi]=0
                    if(v[vx, vi]>=0.7):
                        v[vx, vi]=1
        # testing whether solution is legal
        t1, t2, t3 = 0, 0, 0
        # require total of N-nodes with 1 value
        t1=0;
        for vx in range(N):
            for vi in range(N):
                t1+=v[vx, vi]
        # allow only one node in each row equal 1
        t2=0
        for x in range(N):
            for i in range(N-1):
                for j in range(i+1, N):
                    t2+=np.multiply(v[x, i], v[x, j])
        # allow only one node in each column equal 1
        t3=0
        for i in range(N):
            for x in range(N-1):
                for y in range(x+1, N):
                    t3+=np.multiply(v[x, i], v[y, i])
        # stop the loop after getting the valid solution
        if t1==N and t2==0 and t3==0:
            toend = 1
        else:
            toend = 0
    return(v, ctr)

In [0]:
# calculting the paht cost based on the updated distanvce matrix for each iteration
def path_cost(v):
    x_des = np.zeros([N+1])
    y_des = np.zeros([N+1])
    for j in range(N):
        for i in range(N):
            if v[i,j]==1:
                x_des[j]=x[i]
                y_des[j]=y[i]

    x_des[N]=x_des[0]
    y_des[N]=y_des[0]
    # calculate the total distance
    total_dis=0
    for i in range(N-1):
        total_dis+=np.sqrt((x_des[i]-x_des[i+1])**2+(y_des[i]-y_des[i+1])**2)
    total_dis+=np.sqrt((x_des[N-1]-x_des[0])**2+(y_des[N-1]-y_des[0])**2)
    return(total_dis, x_des, y_des)

In [0]:
# defining the desired city sequence
v_ideal = np.zeros([N,N]) #assuming this as ideal route based on the co-ordinates
#Importing the ideal sequence from the data 
urllib.request.urlretrieve(url = "https://people.sc.fsu.edu/~jburkardt/datasets/tsp/att48_s.txt", filename = "disiredpath.txt")
fname1 = 'disiredpath.txt'
seq= np.loadtxt(fname1)
seq=seq[0:len(seq)-1]
seq=seq.astype(int)
seq = [x - 1 for x in seq]
print(seq)
j=0
for el in seq:
    v_ideal[el, j]=1
    #print(el,j) # Based on the sequence assumed above the distance matrix is obtained
    j+=1
    #print(v_ideal)

[0, 7, 37, 30, 43, 17, 6, 27, 5, 36, 18, 26, 16, 42, 29, 35, 45, 32, 19, 46, 20, 31, 38, 47, 4, 41, 23, 9, 44, 34, 3, 25, 1, 28, 33, 40, 15, 21, 2, 22, 13, 24, 12, 10, 11, 14, 39, 8]


In [0]:
v = np.zeros([N,N])
# desired total distance taken from the euclidean distance matrix
des_td = round(h,4)
count=0
node_list = [] #creating an array where the obtained distance matrix is temporarily stored and updated after each iteration
while True:
    count+=1
    v, steps = hopfield() #calling the network
    td, _, _ = path_cost(v)
    if np.array_equiv(v, v_ideal):
        print("Desired city sequence matrix and distance achieved for {} runs".format(count))
        print(v)
        print("Distance: ", td)
        node_list.append(v)
        break
    elif td<=des_td:
        print("Achieved desired distance for {} runs".format(count))
        print(v)
        print(td)
        node_list.append(v)
    else:
        print("No desired solution, executed for {} steps, total distance {}".format(steps, td))

KeyboardInterrupt: ignored

In [0]:
#creating a function which an represnt the route as shown 
def route_representation(v):
    route = ""
    for j in range(v.shape[1]):
        route += str(np.argmax(v[:, j])) + '->'
    return (route + str(np.argmax(v[:, 0])))

In [0]:
def label(N, x, y):
    labels = np.arange(10) #for the plot generated from the 0btained route the labels for that particular plot is created using references
    for label, x, y in zip(labels, x, y):
        plt.annotate(
            label,
            xy=(x, y), xytext=(-20, 20),
            textcoords='offset points', ha='right', va='bottom',
            bbox=dict(boxstyle='round,pad=0.5', fc='white', alpha=0.5),
            arrowprops=dict(arrowstyle = '->', connectionstyle='arc3,rad=0'))

In [0]:
v = node_list[0]
td, X, Y = path_cost(v)
print("Total distance: ", td)
plt.plot(X, Y,'o-');
label(N, x, y)
print("Given city sequence: {}".format(route_representation(v_ideal)))
print("Obtained city sequence: {}".format(route_representation(v)))

In [0]:
print((time.clock() - start_time)/60, "Minutes")